In [31]:
using Newtonsoft.Json;
using Newtonsoft;
using System.IO;

In [113]:
using System.Linq;
using System.Collections;
using System.Collections.Generic;

class DataIn {
    public string taskName {get;set;}
    public List<Entry> data {get;set;}
}

class Entry {
    public string name {get; set;}
    public string group {get; set;}
    public string discipline {get; set;}
    public int mark {get; set;}
}

class FirstOut {
    public List<StudentGPA> Response {set;get;}

    public FirstOut(List<StudentGPA> e){
        this.Response = e;
    }
}

class SecondOut {
    public List<JProperty> response{get;set;}
}

class SecondEntryResponse {
    public string Discipline {get; set;}
    public double Mark {get; set;}

    public SecondEntryResponse(string d, double m) {
        Discipline = d;
        Mark = m;
    }
}

class StudentGPA {
    public string Cadet {get;set;}
    public double GPA {get;set;}

    public StudentGPA(string c, double gpa) {
        Cadet = c;
        GPA = gpa;
    }
}

string data = File.ReadAllText("test.json");
DataIn dataIn = JsonConvert.DeserializeObject<DataIn>(data);
var outData = new SecondOut();

switch (dataIn.taskName) {
    case "GetStudentsWithHighestGPA":
        var maxMark = dataIn.data
                            .GroupBy(e => e.name)
                            .Select(e => e.Average(e => e.mark))
                            .Max();
        
        var response = new FirstOut(dataIn.data
                        .DistinctBy(e => e.name)
                        .Where(e => dataIn.data.Where(entry => entry.name == e.name).Average(entry => entry.mark) == maxMark)
                        .Select(e => new StudentGPA(e.name, maxMark))
                        .ToList());

        File.WriteAllText("answer.json", JsonConvert.SerializeObject(response));
        break;

    case "CalculateGPAByDiscipline":
        outData.response = 
        dataIn.data
            .GroupBy(e => e.discipline)
            .Select(e => new JProperty(e.First().discipline, e.Average(enrty => enrty.mark)))
            .ToList();        

        File.WriteAllText("answer.json", JsonConvert.SerializeObject(outData));
        break;
}


Error: Newtonsoft.Json.JsonWriterException: Token PropertyName in state ArrayStart would result in an invalid JSON object. Path 'response'.
   at Newtonsoft.Json.JsonWriter.AutoComplete(JsonToken tokenBeingWritten)
   at Newtonsoft.Json.JsonTextWriter.WritePropertyName(String name)
   at Newtonsoft.Json.Linq.JProperty.WriteTo(JsonWriter writer, JsonConverter[] converters)
   at Newtonsoft.Json.Serialization.JsonSerializerInternalWriter.SerializeValue(JsonWriter writer, Object value, JsonContract valueContract, JsonProperty member, JsonContainerContract containerContract, JsonProperty containerProperty)
   at Newtonsoft.Json.Serialization.JsonSerializerInternalWriter.SerializeList(JsonWriter writer, IEnumerable values, JsonArrayContract contract, JsonProperty member, JsonContainerContract collectionContract, JsonProperty containerProperty)
   at Newtonsoft.Json.Serialization.JsonSerializerInternalWriter.SerializeValue(JsonWriter writer, Object value, JsonContract valueContract, Js

Submission#11+DataIn taskName GetStudentsWithHighestGPA data index value 0 Submission#11+Entry name Cadet1 group SC-991 discipline Programming mark 5 1 Submission#11+Entry name Cadet2 group SC-991 discipline Programming mark 3 2 Submission#11+Entry name Cadet1 group SC-991 discipline Algebra mark 5 3 Submission#11+Entry name Cadet2 group SC-991 discipline Algebra mark 4 4 Submission#11+Entry name Cadet3 group SC-992 discipline Programming mark 4 5 Submission#11+Entry name Cadet4 group SC-992 discipline Programming mark 5 6 Submission#11+Entry name Cadet3 group SC-992 discipline Algebra mark 3 7 Submission#11+Entry name Cadet4 group SC-992 discipline Algebra mark 5